# Preference Alignment with Direct Preference Optimization (DPO)

This notebook will guide you through the process of fine-tuning a language model using Direct Preference Optimization (DPO). We will use the SmolLM2-135M-Instruct model which has already been through a SFT training, so it is compatible with DPO.

### Exercise: Aligning SmolLM2 with DPOTrainer</h2>
Take a dataset from the Hugging Face hub and align a model on it.

In [1]:
# Install the requirements in Google Colab
!pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login

login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

pyenv: version `3.11' is not installed (set by /Users/simonamazzarino/Documents/Clearbox-repo/clearbox-ai-academy/Fine-Tuning_LLM/.python-version)


## Import libraries


In [2]:
import torch
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import DPOTrainer, DPOConfig


## Format dataset

DPO requires a preference dataset. The DPOTrainer supports both conversational and standard dataset format. When provided with a conversational dataset, the trainer will automatically apply the chat template to the dataset.

Although the DPOTrainer supports both explicit and implicit prompts, we recommend using explicit prompts. If provided with an implicit prompt dataset, the trainer will automatically extract the prompt from the "chosen" and "rejected" columns. 

In [7]:
# Load dataset

dataset = load_dataset(path="trl-lib/ultrafeedback_binarized", split="train")

In [17]:
dataset

Dataset({
    features: ['chosen', 'rejected', 'score_chosen', 'score_rejected'],
    num_rows: 62135
})

In [18]:
import pandas as pd
df = pd.DataFrame(dataset)

In [19]:
df

,chosen,rejected,score_chosen,score_rejected
0,[{'content': 'Use the pygame library to write ...,[{'content': 'Use the pygame library to write ...,6.0,4.0
1,[{'content': 'QUESTION: She was a horrible pet...,[{'content': 'QUESTION: She was a horrible pet...,8.0,7.5
2,[{'content': 'Detailed Instructions: In this t...,[{'content': 'Detailed Instructions: In this t...,8.0,4.0
3,[{'content': 'write me in game shop system in ...,[{'content': 'write me in game shop system in ...,8.0,8.0
4,[{'content': 'Develop a 10-page research paper...,[{'content': 'Develop a 10-page research paper...,7.0,5.0
...,...,...,...,...
62130,[{'content': 'In this task you are given a lis...,[{'content': 'In this task you are given a lis...,7.0,4.0
62131,[{'content': 'create a job title and descripti...,[{'content': 'create a job title and descripti...,8.5,6.0
62132,[{'content': 'Answer :The fact that people are...,[{'content': 'Answer :The fact that people are...,7.0,6.5
62133,"[{'content': 'In this task, you are given a ha...","[{'content': 'In this task, you are given a ha...",8.0,2.0


In [20]:
df['chosen'][1], df['rejected'][1]

([{'content': "QUESTION: She was a horrible pet owner, she would put a what on her cat?\nOptions:\n- leave outside\n- sharp teeth\n- get wet\n- wool sweater\n- eat vegetables\nANSWER: Wool sweater can be put on a cat. Wool sweater is a vest. Horrible pet owners put on wool sweaters.\nThe answer is wool sweater\n\nQUESTION: Alcohol wasn't allowed at the meeting, so where did the alcoholic put his flask?\nOptions:\n- science lab\n- laboratory\n- coat pocket\n- the meeting room\n- chemistry lab\nANSWER: Person mostly wears coat when he is required to attend meeting and coat have pockets. Alcoholic person keeps alcohol flask with him which he can put in pocket of coat. Alcohol flask are easy to carry and conceal in pocket.\nThe answer is coat pocket\n\nQUESTION: How can a company get things to their customers?\nOptions:\n- mail order\n- carrier pigeon\n- own factory\n- ship goods\n- commit crime\nANSWER: To ship means to send goods to places far away. To send the customers the goods, the c

## Select the model

We will use the SmolLM2-135M-Instruct model which has already been through a SFT training, so it is compatible with DPO.


In [21]:
model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

# Model to fine-tune
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    torch_dtype=torch.float32,
).to(device)
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-DPO"

## Train model with DPO

In [22]:
# Training arguments
training_args = DPOConfig(
    # Training batch size per GPU
    per_device_train_batch_size=4,
    # Number of updates steps to accumulate before performing a backward/update pass
    # Effective batch size = per_device_train_batch_size * gradient_accumulation_steps
    gradient_accumulation_steps=4,
    # Saves memory by not storing activations during forward pass
    # Instead recomputes them during backward pass
    gradient_checkpointing=True,
    # Base learning rate for training
    learning_rate=5e-5,
    # Learning rate schedule - 'cosine' gradually decreases LR following cosine curve
    lr_scheduler_type="cosine",
    # Total number of training steps
    max_steps=200,
    # Disables model checkpointing during training
    save_strategy="no",
    # How often to log training metrics
    logging_steps=1,
    # Directory to save model outputs
    output_dir="smol_dpo_output",
    # Number of steps for learning rate warmup
    warmup_steps=100,
    # Use bfloat16 precision for faster training
    bf16=True,
    # Disable wandb/tensorboard logging
    report_to=None,
    # Keep all columns in dataset even if not used
    remove_unused_columns=False,
    # Enable MPS (Metal Performance Shaders) for Mac devices
    use_mps_device=device == "mps",
    # Model ID for HuggingFace Hub uploads
    hub_model_id=finetune_name,
)

/Users/simonamazzarino/Documents/Clearbox-repo/clearbox-ai-academy/Fine-Tuning_LLM/.venv/lib/python3.11/site-packages/transformers/training_args.py:2243: UserWarning: `use_mps_device` is deprecated and will be removed in version 5.0 of 🤗 Transformers. `mps` device will be used by default if available similar to the way `cuda` device is used.Therefore, no action from user is required. 
  warnings.warn(


In [23]:
trainer = DPOTrainer(
    # The model to be trained
    model=model,
    # Training configuration from above
    args=training_args,
    # Dataset containing preferred/rejected response pairs
    train_dataset=dataset,
    # Tokenizer for processing inputs
    processing_class=tokenizer,
)

In [ ]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")